In [2]:
import re
import sqlite3
from pprint import pprint

import numpy as np
import pandas as pd
import requests as req

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

sns.set(context="talk", style="darkgrid")

pd.options.display.max_rows = 9999
pd.options.display.max_columns = 999

In [3]:
shows = []

# Grab 250 shows at a time...
for start in tqdm(range(0, 16_000, 250)):
    url = f"https://www.ratingraph.com/toplist/shows/?draw=2&columns%5B0%5D%5Bdata%5D=rank&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=false&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=trend&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=false&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=name&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=false&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=start&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=false&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=end&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=false&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=genres&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=false&columns%5B5%5D%5Borderable%5D=false&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=seasons&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=false&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B7%5D%5Bdata%5D=episodes&columns%5B7%5D%5Bname%5D=&columns%5B7%5D%5Bsearchable%5D=false&columns%5B7%5D%5Borderable%5D=true&columns%5B7%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B7%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B8%5D%5Bdata%5D=total_votes&columns%5B8%5D%5Bname%5D=&columns%5B8%5D%5Bsearchable%5D=false&columns%5B8%5D%5Borderable%5D=true&columns%5B8%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B8%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B9%5D%5Bdata%5D=average_votes&columns%5B9%5D%5Bname%5D=&columns%5B9%5D%5Bsearchable%5D=false&columns%5B9%5D%5Borderable%5D=true&columns%5B9%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B9%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B10%5D%5Bdata%5D=average_rating&columns%5B10%5D%5Bname%5D=&columns%5B10%5D%5Bsearchable%5D=false&columns%5B10%5D%5Borderable%5D=true&columns%5B10%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B10%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=asc&start={start}&length=250&search%5Bvalue%5D=&search%5Bregex%5D=false&_=1588128650138"
    data = req.get(url).json()
    shows.extend(data["data"])
    
shows = pd.DataFrame(shows)
shows.head()

100%|██████████| 64/64 [00:25<00:00,  2.55it/s]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249
0,"{'rank': '1', 'trend': '648', 'name': 'Chernob...","{'rank': '2', 'trend': '6,145', 'name': 'Game ...","{'rank': '3', 'trend': '2,991', 'name': 'Black...","{'rank': '4', 'trend': '742', 'name': 'Sherloc...","{'rank': '5', 'trend': '6,979', 'name': 'Break...","{'rank': '6', 'trend': '2,028', 'name': 'The M...","{'rank': '7', 'trend': '1,553', 'name': 'Stran...","{'rank': '8', 'trend': '682', 'name': 'The Wit...","{'rank': '9', 'trend': '5,625', 'name': 'Westw...","{'rank': '10', 'trend': '933', 'name': 'True D...","{'rank': '11', 'trend': '1,365', 'name': 'Rick...","{'rank': '12', 'trend': '4,074', 'name': 'The ...","{'rank': '13', 'trend': '1,060', 'name': 'Love...","{'rank': '14', 'trend': '690', 'name': 'Darede...","{'rank': '15', 'trend': '2,075', 'name': 'Mr. ...","{'rank': '16', 'trend': '268', 'name': 'The Ha...","{'rank': '17', 'trend': '10,888', 'name': 'Bet...","{'rank': '18', 'trend': '1,449', 'name': 'Dark...","{'rank': '19', 'trend': '598', 'name': 'Fargo'...","{'rank': '20', 'trend': '328', 'name': 'Band o...","{'rank': '21', 'trend': '337', 'name': 'Watchm...","{'rank': '22', 'trend': '319', 'name': 'The Bo...","{'rank': '23', 'trend': '1,406', 'name': 'Peak...","{'rank': '24', 'trend': '583', 'name': 'Hannib...","{'rank': '25', 'trend': '1,386', 'name': 'Atta...","{'rank': '26', 'trend': '340', 'name': 'The Pu...","{'rank': '27', 'trend': '1,920', 'name': 'Lost...","{'rank': '28', 'trend': '137', 'name': 'When T...","{'rank': '29', 'trend': '1,944', 'name': 'Doct...","{'rank': '30', 'trend': '129', 'name': 'Firefl...","{'rank': '31', 'trend': '558', 'name': 'The Le...","{'rank': '32', 'trend': '5,315', 'name': 'Mone...","{'rank': '33', 'trend': '662', 'name': 'Narcos...","{'rank': '34', 'trend': '546', 'name': 'Mindhu...","{'rank': '35', 'trend': '1,030', 'name': 'Luci...","{'rank': '36', 'trend': '1,017', 'name': 'Dext...","{'rank': '37', 'trend': '47', 'name': 'Taboo',...","{'rank': '38', 'trend': '229', 'name': 'Twin P...","{'rank': '39', 'trend': '188', 'name': 'Sense8...","{'rank': '40', 'trend': '235', 'name': 'Big Li...","{'rank': '41', 'trend': '31', 'name': 'Spartac...","{'rank': '42', 'trend': '85', 'name': 'The Nig...","{'rank': '43', 'trend': '638', 'name': 'House ...","{'rank': '44', 'trend': '4,612', 'name': 'Frie...","{'rank': '45', 'trend': '102', 'name': 'Dekalo...","{'rank': '46', 'trend': '609', 'name': 'The 10...","{'rank': '47', 'trend': '67', 'name': 'The Def...","{'rank': '48', 'trend': '325', 'name': 'Twin P...","{'rank': '49', 'trend': '795', 'name': 'Person...","{'rank': '50', 'trend': '1,736', 'name': 'Arro...","{'rank': '51', 'trend': '1,321', 'name': 'The ...","{'rank': '52', 'trend': '1,582', 'name': 'Viki...","{'rank': '53', 'trend': '591', 'name': 'The Ha...","{'rank': '54', 'trend': '92', 'name': 'Dracula...","{'rank': '55', 'trend': '118', 'name': 'Bodygu...","{'rank': '56', 'trend': '30', 'name': 'Planet ...","{'rank': '57', 'trend': '3,480', 'name': 'Supe...","{'rank': '58', 'trend': '1,824', 'name': 'The ...","{'rank': '59', 'trend': '318', 'name': 'Sparta...","{'rank': '60', 'trend': '906', 'name':

In [8]:
shows.head()

,rank,trend,name,start,end,genres,seasons,episodes,total_votes,average_votes,average_rating,poster,path
0,1,648,Chernobyl,2019,2019,"Drama, History, Thriller",1,5,"219,554","43,911",9.6,/assets/images/shows/chernobyl-poster-67251.jpg,/tv-shows/chernobyl-ratings-67251/
1,2,"6,145",Game of Thrones,2011,2019,"Action, Drama, Adventure",8,73,"3,484,981","47,739",8.8,/assets/images/shows/game-of-thrones-poster-26...,/tv-shows/game-of-thrones-ratings-26649/
2,3,"2,991",Black Mirror,2011,-,"Drama, Sci-fi, Thriller",5,22,"839,953","38,180",7.9,/assets/images/shows/black-mirror-poster-42161...,/tv-shows/black-mirror-ratings-42161/
3,4,742,Sherlock,2010,-,"Drama, Crime, Mystery",4,12,"309,700","25,808",8.9,/assets/images/shows/sherlock-poster-35707.jpg,/tv-shows/sherlock-ratings-35707/
4,5,"6,979",Breaking Bad,2008,2013,"Drama, Crime, Thriller",5,62,"1,169,385","18,861",9.0,/assets/images/shows/breaking-bad-poster-26165...,/tv-shows/breaking-bad-ratings-26165/


In [11]:
shows["episodes"] = shows.episodes.str.replace(",", "").astype(int)
shows["total_votes"] = shows.total_votes.str.replace(",", "").astype(int)
shows["average_votes"] = shows.average_votes.str.replace(",", "").astype(int)
shows.head()

,rank,trend,name,start,end,genres,seasons,episodes,total_votes,average_votes,average_rating,poster,path
0,1,648,Chernobyl,2019,2019,"Drama, History, Thriller",1,5,219554,43911,9.6,/assets/images/shows/chernobyl-poster-67251.jpg,/tv-shows/chernobyl-ratings-67251/
1,2,"6,145",Game of Thrones,2011,2019,"Action, Drama, Adventure",8,73,3484981,47739,8.8,/assets/images/shows/game-of-thrones-poster-26...,/tv-shows/game-of-thrones-ratings-26649/
2,3,"2,991",Black Mirror,2011,-,"Drama, Sci-fi, Thriller",5,22,839953,38180,7.9,/assets/images/shows/black-mirror-poster-42161...,/tv-shows/black-mirror-ratings-42161/
3,4,742,Sherlock,2010,-,"Drama, Crime, Mystery",4,12,309700,25808,8.9,/assets/images/shows/sherlock-poster-35707.jpg,/tv-shows/sherlock-ratings-35707/
4,5,"6,979",Breaking Bad,2008,2013,"Drama, Crime, Thriller",5,62,1169385,18861,9.0,/assets/images/shows/breaking-bad-poster-26165...,/tv-shows/breaking-bad-ratings-26165/


In [13]:
shows.episodes.describe([0.1, 0.25, 0.5, 0.75, 0.9])

count    16000.000000
mean        27.794062
std         70.388158
min          2.000000
10%          3.000000
25%          5.000000
50%         10.000000
75%         25.000000
90%         64.000000
max       3750.000000
Name: episodes, dtype: float64

In [16]:
import re

shows["show_id"] = shows.path.apply(lambda x: re.findall("(\d+)\/$", x)[0])
shows.head()

,rank,trend,name,start,end,genres,seasons,episodes,total_votes,average_votes,average_rating,poster,path,show_id
0,1,648,Chernobyl,2019,2019,"Drama, History, Thriller",1,5,219554,43911,9.6,/assets/images/shows/chernobyl-poster-67251.jpg,/tv-shows/chernobyl-ratings-67251/,67251
1,2,"6,145",Game of Thrones,2011,2019,"Action, Drama, Adventure",8,73,3484981,47739,8.8,/assets/images/shows/game-of-thrones-poster-26...,/tv-shows/game-of-thrones-ratings-26649/,26649
2,3,"2,991",Black Mirror,2011,-,"Drama, Sci-fi, Thriller",5,22,839953,38180,7.9,/assets/images/shows/black-mirror-poster-42161...,/tv-shows/black-mirror-ratings-42161/,42161
3,4,742,Sherlock,2010,-,"Drama, Crime, Mystery",4,12,309700,25808,8.9,/assets/images/shows/sherlock-poster-35707.jpg,/tv-shows/sherlock-ratings-35707/,35707
4,5,"6,979",Breaking Bad,2008,2013,"Drama, Crime, Thriller",5,62,1169385,18861,9.0,/assets/images/shows/breaking-bad-poster-26165...,/tv-shows/breaking-bad-ratings-26165/,26165


In [17]:
conn = sqlite3.connect("imdb.db")

In [18]:
columns = [
    "show_id",
    "name",
    "start",
    "end",
    "genres",
    "seasons",
    "episodes",
    "total_votes",
    "average_votes",
    "average_rating",
    "trend",
    "rank",
    "poster",
    "path",
]

shows[columns].to_sql("show", conn, index=False, if_exists="replace")